In [13]:
import math
import numpy as np

# Constants
rho0 = 1.225  # Sea level air density (kg/m^3)
h = [0, 5000, 10000, 0]  # Altitudes in meters (takeoff, climb, cruise, landing)
Cd0 = 0.02  # Parasitic drag coefficient for a BWB
delta = 0.005  # Induced drag correction factor
AR = 9.0  # Aspect ratio
CL = [0.9, 0.7, 0.5, 1.6]  # Lift coefficients for phases (higher for blended wing body)
S = 164.9  # Wing area (m^2)
W = 471511.32  # Aircraft weight (N)
gamma = 5  # Climb angle in degrees
v = [70, 140, 250, 56]  # Speeds (m/s) for each phase

# Drag coefficients
Cd = np.array([Cd0 + ((cl ** 2) / (math.pi * AR)) * (1 + delta) for cl in CL])

# Density function (ISA model)
def rho(h):
    rho0 = 1.225
    sea_level_temp = 288.15
    lapse_rate = -0.0065
    R = 287.05
    g = 9.80665
    tropopause_height = 11000

    if isinstance(h, (list, np.ndarray)):
        return np.array([rho(alt) for alt in h])
    if h <= tropopause_height:
        temperature = sea_level_temp + lapse_rate * h
        return rho0 * (temperature / sea_level_temp) ** (-g / (lapse_rate * R))
    else:
        temperature = sea_level_temp + lapse_rate * tropopause_height
        exp_factor = (-g * (h - tropopause_height)) / (R * temperature)
        return rho0 * (temperature / sea_level_temp) ** (-g / (lapse_rate * R)) * math.exp(exp_factor)

# Dynamic pressure calculation
def dynamic_pressure(V, h):
    densities = rho(h)
    return 0.5 * densities * np.array(V)**2

# Thrust required for steady flight
def thrust_required(Cd, q, S):
    return Cd * q * S

# Additional thrust for climb
def thrust_required_climb(W, gamma):
    gamma_rad = math.radians(gamma)
    return W * math.sin(gamma_rad)

# Phase calculations
q = dynamic_pressure(v, h)  # Dynamic pressure for each phase
T = [thrust_required(Cd[i], q[i], S) for i in range(len(v))]
T[1] += thrust_required_climb(W, gamma)  # Add climb-specific thrust

# Results
phases = ["Takeoff", "Climb", "Cruise", "Landing"]
for i, phase in enumerate(phases):
    print(f"{phase} thrust: {T[i]:.2f} N")


Takeoff thrust: 24147.03 N
Climb thrust: 80584.68 N
Cruise thrust: 47574.44 N
Landing thrust: 35156.29 N


In [ ]:
theta = 0.752  # Temperature ratio (cruise altitude temp / sea level temp)
M = 0.85       # Mach number
n = 0.8
TSFC0 = 16974.41  # Takeoff thrust

# Correcting the exponentiation operator
TSFC = TSFC0 * (theta ** 0.5) * ((1 + M) ** n)

print(f"Thrust specific fuel consumption: {TSFC:.2f}")
#this still has some problems with it 







Thrust specific fuel consumption: 24079.14
